In [13]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/nigeria-crop-yield/extracted_data.csv
/kaggle/input/nigeria-crop-yield/crop_production.csv
/kaggle/input/nigeria-crop-yield/nigeria_crops.csv
/kaggle/input/potato-production-in-plateau-state-2023/ps_pvcp_production_data_sept_2023.csv
/kaggle/input/potato-production-in-plateau-state-2023/ps_pvcp_production_data_sept_2023--dirty.csv
/kaggle/input/nigeria-agri-econ-indicators-1950-2023/nigeria_agricultural_economic_indicators_1950_2023.csv
/kaggle/input/ondo-state-nigeria-cocoa-cassava-and-weather-data/Rainfall.txt
/kaggle/input/ondo-state-nigeria-cocoa-cassava-and-weather-data/Min_tempt.txt
/kaggle/input/ondo-state-nigeria-cocoa-cassava-and-weather-data/COCOA DATA.xlsx
/kaggle/input/ondo-state-nigeria-cocoa-cassava-and-weather-data/Max_tempt.txt
/kaggle/input/ondo-state-nigeria-cocoa-cassava-and-weather-data/CASSAVA.csv


**GOOGLE KEYS AND DEPENDENCIES **

In [14]:
# CELL 1 — FULL SETUP (run once)
!pip install google-adk google-generativeai gtts --quiet

import os, logging, requests, pandas as pd, warnings
warnings.filterwarnings('ignore')
from kaggle_secrets import UserSecretsClient
from google.adk.agents import Agent
from google.adk.models.google_llm import Gemini
from google.adk.runners import InMemoryRunner
from google.adk.tools import FunctionTool
from gtts import gTTS
import IPython.display as ipd

os.environ["GOOGLE_API_KEY"] = UserSecretsClient().get_secret("GOOGLE_API_KEY")
logging.getLogger("google_genai.types").setLevel(logging.ERROR)
print("Setup complete!")

Setup complete!


**Nigerian  Crop Data Sets**

In [15]:
# CELL 2 — FIXED: Auto-Load All Available Datasets + Robust Tool
import pandas as pd
from google.adk.tools import FunctionTool

# Step 1: List ALL available files in /kaggle/input (run this to see what's attached)
print("=== AVAILABLE DATASETS FILES ===")
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        if filename.endswith('.csv'):
            print(f"📁 Found: {os.path.join(dirname, filename)}")

# Step 2: Auto-load ALL CSVs (no hard-coded names)
master_df = pd.DataFrame()
input_dirs = ['/kaggle/input/potato-production-in-plateau-state-2023', '/kaggle/input/nigeria-crop-yield', 
              '/kaggle/input/ondo-state-nigeria-cocoa-cassava-and-weather-data', '/kaggle/input/nigeria-agric-exports-2020-2023']

for dir_path in input_dirs:
    try:
        files = [f for f in os.listdir(dir_path) if f.endswith('.csv')]
        if not files:
            print(f"⚠️ No CSV in {dir_path}")
            continue
        # Load the first (main) CSV in each directory
        csv_path = os.path.join(dir_path, files[0])
        df = pd.read_csv(csv_path)
        print(f"✅ Loaded {csv_path} ({df.shape[0]} rows, columns: {list(df.columns)})")
        
        # Auto-map columns based on dataset slug (flexible!)
        if 'potato' in dir_path.lower():
            df['State'] = 'Plateau'
            df['Crop'] = 'Potato'
            df['Soil_Type'] = df.get('soil_type', 'Loamy')  # Adapt if column exists
        elif 'crop-yield' in dir_path.lower():
            df['State'] = 'Nigeria'
            df['Crop'] = df.get('crop', df.get('crop_name', 'Unknown Crop'))
            df['Soil_Type'] = 'Varies'
        elif 'ondo' in dir_path.lower():
            df['State'] = 'Ondo'
            df['Crop'] = df.get('crop', 'Cassava/Cocoa')
            df['Soil_Type'] = 'Sandy Loam'
        else:  # Exports or other
            df['State'] = 'Nigeria'
            df['Crop'] = df.get('crop', 'Export Crop')
            df['Soil_Type'] = 'Varies'
        
        # Add defaults for missing columns
        df['Common_Diseases'] = df.get('diseases', df.get('common_diseases', 'Blight, Rot, Mosaic'))
        df['Advice'] = df.get('advice', 'Use certified seeds, good drainage, monitor weather')
        df['Yield_tons_ha'] = pd.to_numeric(df.get('yield', df.get('yield_tons_ha', df.get('yield_kg_ha', 10000))) / 1000, errors='coerce')
        
        # Select/rename key columns
        key_cols = ['State', 'Crop', 'Soil_Type', 'Common_Diseases', 'Advice', 'Yield_tons_ha']
        available_cols = [col for col in key_cols if col in df.columns or col.lower() in [c.lower() for c in df.columns]]
        df = df[available_cols + [col for col in df.columns if col.lower() in ['year', 'lga', 'area_ha']]]  # Keep extras like year/LGA
        
        master_df = pd.concat([master_df, df], ignore_index=True)
    except Exception as e:
        print(f"⚠️ Error loading {dir_path}: {e}")

# Fallback if no real data loaded
if master_df.empty or len(master_df) == 0:
    print("⚠️ No real datasets found — using backup")
    master_df = pd.DataFrame({
        'State': ['Plateau', 'Nigeria', 'Ondo'],
        'Crop': ['Potato', 'Rice', 'Cassava'],
        'Soil_Type': ['Loamy', 'Varies', 'Sandy Loam'],
        'Common_Diseases': ['Blight, Rot', 'Blast', 'Mosaic Virus'],
        'Advice': ['Good drainage + fungicide', 'Certified seeds', 'Rotate crops'],
        'Yield_tons_ha': [15.0, 3.5, 18.0]
    })

# Clean and dedupe
master_df = master_df.fillna('N/A').drop_duplicates(subset=['State', 'Crop']).reset_index(drop=True)
print(f"\n✅ FINAL MERGED DATASET: {len(master_df)} rows")
print("Sample rows:")
print(master_df.head(3))
print(f"Crops covered: {master_df['Crop'].unique()}")

# FIXED Tool: Flexible search (case-insensitive, partial matches)
def lookup_agri_data(state: str, crop: str = "") -> str:
    # Flexible matching
    mask = master_df['State'].str.contains(state, case=False, na=False)
    if crop:
        mask &= master_df['Crop'].str.contains(crop, case=False, na=False)
    result = master_df[mask]
    if result.empty:
        return f"No data for {crop or 'crop'} in {state}. Try national averages."
    
    row = result.iloc[0]
    return f"""FACTS ({row['State']}, {row['Crop']}):
• Soil: {row['Soil_Type']}
• Diseases: {row['Common_Diseases']}
• Advice: {row['Advice']}
• Yield: {row['Yield_tons_ha']:.1f} tons/ha (real data)"""

agri_tool = FunctionTool(lookup_agri_data)
print("\n✅ Agri Tool Ready (handles all datasets)!")

=== AVAILABLE DATASETS FILES ===
📁 Found: /kaggle/input/nigeria-crop-yield/extracted_data.csv
📁 Found: /kaggle/input/nigeria-crop-yield/crop_production.csv
📁 Found: /kaggle/input/nigeria-crop-yield/nigeria_crops.csv
📁 Found: /kaggle/input/potato-production-in-plateau-state-2023/ps_pvcp_production_data_sept_2023.csv
📁 Found: /kaggle/input/potato-production-in-plateau-state-2023/ps_pvcp_production_data_sept_2023--dirty.csv
📁 Found: /kaggle/input/nigeria-agri-econ-indicators-1950-2023/nigeria_agricultural_economic_indicators_1950_2023.csv
📁 Found: /kaggle/input/ondo-state-nigeria-cocoa-cassava-and-weather-data/CASSAVA.csv
✅ Loaded /kaggle/input/potato-production-in-plateau-state-2023/ps_pvcp_production_data_sept_2023.csv (1783 rows, columns: ['id', 'today', 'head_of_household', 'lga_of_residence', 'village', 'survey_day', 'survey_month', 'time_taken', 'birth_year', 'gender', 'marital_status', 'household_size', 'educational_level', 'living_arrangement', 'roof_type', 'coop_society_member', 

### WEATHER WARNING TOOL ####

In [16]:
# CELL 3 — Weather + Market + Pidgin
market_prices = {("plateau","potato"):"₦850–950/kg → Very high! Plant now!"}

def to_pidgin(text):
    rep = {"Ensure":"Make sure", "drainage":"water comot", "Use":"Use", "high":"high"}
    for e,p in rep.items():
        text = text.replace(e,p).replace(e.lower(),p.lower())
    return text + " o!"

coords = {"plateau": (9.8965, 8.8583), "ondo": (7.25, 4.85), "nigeria": (9.08, 8.67)}

def get_weather_warning(state: str) -> str:
    lat, lon = coords.get(state.lower(), coords["nigeria"])
    try:
        url = f"https://api.open-meteo.com/v1/forecast?latitude={lat}&longitude={lon}&daily=precipitation_sum"
        rain = requests.get(url, timeout=8).json()["daily"]["precipitation_sum"][0]
        if rain > 10: return f"HEAVY RAIN ({rain}mm) → Check drainage NOW!"
        elif rain > 3: return f"Some rain ({rain}mm) → Watch your field"
        else: return f"Good weather ({rain}mm) → Safe to plant!"
    except:
        return "Weather check failed."

weather_tool = FunctionTool(get_weather_warning)

In [17]:
# CELL 4 — FIXED MODEL: Use gemini-2.5-flash (stable & free)

def get_text(events):
    """Bullet-proof text extractor — works with ALL Gemini/ADK versions"""
    text_parts = []
    for event in reversed(events):
        if hasattr(event, "message") and event.message:
            msg = event.message
            if hasattr(msg, "text") and msg.text:
                text_parts.append(msg.text.strip())
            if hasattr(msg, "content") and msg.content:
                for part in msg.content:
                    if hasattr(part, "text") and part.text:
                        text_parts.append(part.text.strip())
        if hasattr(event, "content") and event.content:
            for part in event.content:
                if hasattr(part, "text") and part.text:
                    text_parts.append(part.text.strip())
    result = "\n".join(text_parts).strip()
    return result if result else "No facts found."

# RESEARCHER – uses stable model
researcher = Agent(
    model=Gemini(model="gemini-2.5-flash"),  # ← FIXED: Stable, free, no 404
    name="Data_Expert",
    tools=[agri_tool, weather_tool],
    instruction="""
    You MUST use both tools (lookup_agri_data and get_weather_warning).
    Return ONLY the raw facts and weather report.
    Never say hello, never explain, never add extra words.
    Example output:
    FACTS (Plateau, Potato):
    • Yield: 15.2 tons/ha
    • Diseases: Blight, Rot
    • Advice: Good drainage + fungicide
    Good weather (0mm) → Safe to plant!
    """
)

# EXTENSION WORKER – warm Pidgin brother
extension_worker = Agent(
    model=Gemini(model="gemini-2.5-flash"),  # ← FIXED: Stable, free, no 404
    name="Field_Brother",
    instruction="""
    You are a joyful Nigerian extension officer speaking only warm Nigerian Pidgin.
    Use EVERY fact you are given.
    Make the farmer feel excited and confident.
    Always end with encouragement + short prayer.
    """
)

print("Agents + extractor FIXED — using stable gemini-2.5-flash!")

Agents + extractor FIXED — using stable gemini-2.5-flash!


**VOICE OUTPUT**

In [18]:
# CELL 5 — FINAL VOICE (you will now hear the FULL long advice!)

def speak_advice(text):
    # Make it sound natural when spoken
    clean = text.replace("•", " - ").replace("\n", ". ").replace("  ", " ").strip()
    tts = gTTS(text=clean, lang='en', slow=False)
    tts.save("full_advice.mp3")
    return ipd.Audio("full_advice.mp3", autoplay=True)

async def ask_farmer(query):
    print(f"Farmer: {query}\n" + "="*85)
    
    # Researcher → gets real facts + weather
    events1 = await InMemoryRunner(agent=researcher).run_debug(query)
    facts = get_text(events1)
    print(f"FACTS + WEATHER:\n{facts}\n" + "-"*85)
    
    # Market
    market = market_prices.get(("plateau", "potato"), "₦850–950/kg → Very high! Plant now!")
    
    # Extension worker → full warm advice
    prompt = f"""
    Farmer's question: {query}

    Facts + today's weather:
    {facts}

    Market price:
    {market}

    Answer only in very warm, joyful Nigerian Pidgin.
    End with a short prayer or blessing.
    """
    events2 = await InMemoryRunner(agent=extension_worker).run_debug(prompt)
    advice = get_text(events2)
    advice = to_pidgin(advice)  # adds the final "o!"
    
    print(f"FULL ADVICE TO FARMER:\n{advice}\n" + "="*85)
    
    # SPEAK THE ENTIRE ADVICE ALOUD
    return speak_advice(advice)

# TEST — YOU WILL HEAR THE LONG BEAUTIFUL ADVICE + PRAYER!
await ask_farmer("I want to grow Potatoes in Plateau state but I am worried about diseases.")

Farmer: I want to grow Potatoes in Plateau state but I am worried about diseases.

 ### Created new session: debug_session_id

User > I want to grow Potatoes in Plateau state but I am worried about diseases.
Data_Expert > FACTS (Plateau, Potato):
• Soil: Loamy
• Diseases: Blight, Rot, Mosaic
• Advice: Use certified seeds, good drainage, monitor weather
• Yield: 10.0 tons/ha (real data)
Good weather (0.0mm) → Safe to plant!
FACTS + WEATHER:
No facts found.
-------------------------------------------------------------------------------------

 ### Created new session: debug_session_id

User > 
    Farmer's question: I want to grow Potatoes in Plateau state but I am worried about diseases.

    Facts + today's weather:
    No facts found.

    Market price:
    ₦850–950/kg → Very high! Plant now!

    Answer only in very warm, joyful Nigerian Pidgin.
    End with a short prayer or blessing.
    
Field_Brother > Ah, my good farmer! God bless your hustle! I hear say you wan plant potato for